In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-iulrg460
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-iulrg460
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=35d568fe984f9231fb7d95428ddae2561c33485bf43f8ec58f6ddcabbbfa31a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-h03ecq21/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu 
#include <iostream> 
int main() 
{ 
    std::cout << "Welcome!\n"; 
    return 0; 
} 

Welcome!



In [ ]:
%%cu
#include <stdio.h>
#define N 10

__global__
void add(int *a, int *b) {
    int i = blockIdx.x;
	printf("Hello cuda from thread %d\n", i);
    b[i] = 2*a[i];
}

int main() {
    int ha[N], hb[N];
    int *da, *db;
    cudaMalloc((void **)&da, N*sizeof(int));
    cudaMalloc((void **)&db, N*sizeof(int));
    for (int i = 0; i<N; ++i) {
        ha[i] = i;
    }
    cudaMemcpy(da, ha, N*sizeof(int), cudaMemcpyHostToDevice);
    add<<<N, 1>>>(da, db);
    cudaMemcpy(hb, db, N*sizeof(int), cudaMemcpyDeviceToHost);

    for (int i = 0; i<N; ++i) {
        printf("%d\n", hb[i]);
    }

    cudaFree(da);
    cudaFree(db);

    return 0;
}

In [ ]:
%%cu
#include <stdio.h>
#include <iostream>
#include <stdlib.h>
typedef struct {
  int width;
  int height;
  float* elements;
} Matrix;

#define BLOCK_SIZE 2

__global__ void MatMulKernel(Matrix A, Matrix B, Matrix C)
{
  float Cvalue = 0;
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  for (int e = 0; e < A.width; ++e)
    Cvalue += A.elements[row * A.width + e] * B.elements[e * B.width + col];
  C.elements[row * C.width + col] = Cvalue;
}

void MatMul(const Matrix A, const Matrix B, Matrix C)
{

  Matrix d_A;
  d_A.width = A.width; d_A.height = A.height;
  size_t size = A.width * A.height * sizeof(float);
  cudaMalloc(&d_A.elements, size);
  cudaMemcpy(d_A.elements, A.elements, size, cudaMemcpyHostToDevice);
 
  Matrix d_B;
  d_B.width = B.width; d_B.height = B.height;
  size = B.width * B.height * sizeof(float);
  cudaMalloc(&d_B.elements, size);
  cudaMemcpy(d_B.elements, B.elements, size, cudaMemcpyHostToDevice);

  Matrix d_C;
  d_C.width = C.width; d_C.height = C.height;
  size = C.width * C.height * sizeof(float);
  cudaMalloc(&d_C.elements, size);

  dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
  dim3 dimGrid(B.width / dimBlock.x, A.height / dimBlock.y);
  MatMulKernel<<<dimGrid, dimBlock>>>(d_A, d_B, d_C);
  cudaMemcpy(C.elements, d_C.elements, size, cudaMemcpyDeviceToHost);
  cudaFree(d_A.elements);
  cudaFree(d_B.elements);
  cudaFree(d_C.elements);
}

int main() {
  Matrix a,b,c;
  a.width = BLOCK_SIZE*2;
  a.height = BLOCK_SIZE*3;
  cudaMallocHost((void**)&a.elements,sizeof(float)*a.width*a.height);
  for (int i = 0; i < a.height; i++){
      for(int j = 0; j < a.width; j++){
          a.elements[i*a.width+j] = rand()%20 - 10;
      }
  }
  b.width = BLOCK_SIZE*3;
  b.height = BLOCK_SIZE*2;
  cudaMallocHost((void**)&b.elements,sizeof(float)*b.width*b.height);
  for (int i = 0; i < b.height; i++){
      for(int j = 0; j < b.width; j++){
          b.elements[i*b.width+j] = rand()%20 - 10;
      }
  }
  c.width = BLOCK_SIZE*3;
  c.height = BLOCK_SIZE*3;
  cudaMallocHost((void**)&c.elements,sizeof(float)*c.width*c.height);
  MatMul(a,b,c);
  for (int i = 0; i < c.height; i++){
      for(int j = 0; j < c.width; j++){
          printf("%lf ",c.elements[i*c.width+j]);
      }
      printf("\n");
  }
}

UsageError: Cell magic `%%cu` not found.


In [ ]:
%%cu
#include <stdio.h>
#include <iostream>
#include <stdlib.h>
typedef struct {
  int width;
  int height;
  int stride;
  float* elements;
} Matrix;
#define BLOCK_SIZE 16

__device__ float GetElement(const Matrix A, int row, int col)
{
  return A.elements[row * A.stride + col];
}

__device__ void SetElement(Matrix A, int row, int col,float value)
{
  A.elements[row * A.stride + col] = value;
}

__device__ Matrix GetSubMatrix(Matrix A, int row, int col)
{
  Matrix Asub;
  Asub.width = BLOCK_SIZE;
  Asub.height = BLOCK_SIZE;
  Asub.stride = A.stride;
  Asub.elements = &A.elements[A.stride * BLOCK_SIZE * row + BLOCK_SIZE * col];
  return Asub;
}

__global__ void MatMulKernel(const Matrix, const Matrix, Matrix);

void MatMul(const Matrix A, const Matrix B, Matrix C)
{
  // Load A and B to device memory
  Matrix d_A;
  d_A.width = d_A.stride = A.width; d_A.height = A.height;
  size_t size = A.width * A.height * sizeof(float);
  cudaMalloc(&d_A.elements, size);
  cudaMemcpy(d_A.elements, A.elements, size,
  cudaMemcpyHostToDevice);
  Matrix d_B;
  d_B.width = d_B.stride = B.width; d_B.height = B.height;
  size = B.width * B.height * sizeof(float);
  cudaMalloc(&d_B.elements, size);
  cudaMemcpy(d_B.elements, B.elements, size,
  cudaMemcpyHostToDevice);
  // Allocate C in device memory
  Matrix d_C;
  d_C.width = d_C.stride = C.width; d_C.height = C.height;
  size = C.width * C.height * sizeof(float);
  cudaMalloc(&d_C.elements, size);
  // Invoke kernel
  dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
  dim3 dimGrid(B.width / dimBlock.x, A.height / dimBlock.y);
  MatMulKernel<<<dimGrid, dimBlock>>>(d_A, d_B, d_C);
  // Read C from device memory
  cudaMemcpy(C.elements, d_C.elements, size,
  cudaMemcpyDeviceToHost);
  // Free device memory
  cudaFree(d_A.elements);
  cudaFree(d_B.elements);
  cudaFree(d_C.elements);
}
// Matrix multiplication kernel called by MatMul()
__global__ void MatMulKernel(Matrix A, Matrix B, Matrix C)
{

  int blockRow = blockIdx.y;
  int blockCol = blockIdx.x;

  Matrix Csub = GetSubMatrix(C, blockRow, blockCol);

  float Cvalue = 0;
  int row = threadIdx.y;
  int col = threadIdx.x;
  for (int m = 0; m < (A.width / BLOCK_SIZE); ++m) {
    Matrix Asub = GetSubMatrix(A, blockRow, m);

    Matrix Bsub = GetSubMatrix(B, m, blockCol);

    __shared__ float As[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ float Bs[BLOCK_SIZE][BLOCK_SIZE];

    As[row][col] = GetElement(Asub, row, col);
    Bs[row][col] = GetElement(Bsub, row, col);

    __syncthreads();

    for (int e = 0; e < BLOCK_SIZE; ++e)
      Cvalue += As[row][e] * Bs[e][col];

    __syncthreads();
  }

  SetElement(Csub, row, col, Cvalue);
}

int main() {
  Matrix a,b,c;
  a.width = BLOCK_SIZE*2;
  a.height = BLOCK_SIZE*3;
  cudaMallocHost((void**)&a.elements,sizeof(float)*a.width*a.height);
  for (int i = 0; i < a.height; i++){
      for(int j = 0; j < a.width; j++){
          a.elements[i*a.width+j] = rand()%20 - 10;
      }
  }
  b.width = BLOCK_SIZE*3;
  b.height = BLOCK_SIZE*2;
  cudaMallocHost((void**)&b.elements,sizeof(float)*b.width*b.height);
  for (int i = 0; i < b.height; i++){
      for(int j = 0; j < b.width; j++){
          b.elements[i*b.width+j] = rand()%20 - 10;
      }
  }
  c.width = BLOCK_SIZE*3;
  c.height = BLOCK_SIZE*3;
  cudaMallocHost((void**)&c.elements,sizeof(float)*c.width*c.height);
  MatMul(a,b,c);
  for (int i = 0; i < c.height; i++){
      for(int j = 0; j < c.width; j++){
          printf("%lf ",c.elements[i*c.width+j]);
      }
      printf("\n");
  }
}

186.000000 -27.000000 389.000000 135.000000 -137.000000 244.000000 185.000000 242.000000 -275.000000 -13.000000 225.000000 124.000000 -38.000000 -95.000000 74.000000 -150.000000 -28.000000 -222.000000 -166.000000 -178.000000 232.000000 62.000000 91.000000 185.000000 176.000000 -17.000000 -208.000000 -159.000000 -78.000000 -23.000000 280.000000 72.000000 156.000000 243.000000 280.000000 209.000000 52.000000 -238.000000 536.000000 -188.000000 397.000000 73.000000 -133.000000 136.000000 222.000000 -116.000000 -135.000000 14.000000 
149.000000 -464.000000 198.000000 124.000000 329.000000 -74.000000 53.000000 -121.000000 -369.000000 -221.000000 -40.000000 -307.000000 29.000000 -141.000000 98.000000 172.000000 273.000000 21.000000 -117.000000 220.000000 -319.000000 516.000000 -61.000000 27.000000 64.000000 -34.000000 -20.000000 -215.000000 -159.000000 -2.000000 -127.000000 -309.000000 38.000000 -37.000000 -170.000000 105.000000 -76.000000 99.000000 124.000000 183.000000 206.000000 -81.000000

In [38]:
%%cu
#include <stdio.h>
#include <iostream>
#include <stdlib.h>

#define BLOCK_SIZE 16

__global__ void kernel(int* data, int size, int* answer)
{
  *answer = 0;
  int Cvalue = 0;
  int step = size / (gridDim.x * blockDim.x);
  int id = blockIdx.x * blockDim.x + threadIdx.x;
  int start = id * step;
  int end = (start + step > size) ? size : start + step;
  for (int e = start; e < end; ++e) {
    Cvalue += data[e];
  }
  atomicAdd(answer,Cvalue);
}
int main() {
  int* a,*d_a;
  int* results, *d_res;
  int nStreams = 5;
  int size = BLOCK_SIZE*BLOCK_SIZE*1000;
  int streamSize = size / nStreams;
  cudaMallocHost((void**)&a,sizeof(int)*size);
  cudaMallocHost((void**)&results,sizeof(int)*nStreams);
  for (int i = 0; i < size; i++){
      a[i] = rand()%20 - 10; 
  }
  cudaMalloc((void**)&d_a,sizeof(int)*streamSize);
  cudaMalloc((void**)&d_res,sizeof(int)*nStreams);
  cudaStream_t stream[5];
  for (int i = 0; i < nStreams; i ++)
  {
    cudaStreamCreate(&stream[i]);
  }
  for (int i = 0; i < nStreams; i++) 
  { 
    int offset = streamSize * i;
    cudaMemcpyAsync(d_a, &a[offset], sizeof(int)*streamSize, cudaMemcpyHostToDevice, stream[i]);
    kernel<<<BLOCK_SIZE, BLOCK_SIZE, 0, stream[i]>>>(d_a, streamSize, &d_res[i]);
    cudaMemcpyAsync(&results[i], &d_res[i], sizeof(int), cudaMemcpyDeviceToHost, stream[i]);
  }
  cudaDeviceSynchronize();
  int res = 0;
  for (int i = 0; i < nStreams; i++) {
      res += results[i];
  }
  for (int i = 0; i < nStreams; i++)
  {
    cudaStreamDestroy(stream[i]);
  }
  cudaFree(d_a);
  cudaFree(d_res);
  cudaFree(a);
  cudaFree(results);
  printf("%d\n",res);
}

-126885



In [ ]:
!ls "/content/drive/My Drive"

 15998112035122737757334434944663.jpg
 15998112648576631269851627752573.jpg
 2_5199795001210439887.gdoc
 2_5251585159974619758.py.gdoc
'A cloud brokerage approach for solving the resource management problem in multi-cloud environments.gslides'
 bestfit_table.gslides
'Colab Notebooks'
 Cosine.gslides
'eLearning Platform.gantter'
 Obrazets_Titulnogo_Lista_I_Forma_Otcheta_proizvod_op_Pmi.gdoc
 out_p0.gdoc
 out_ub_part3_p0.gdoc
'report 22.03.2017-29.03.2017.html.gdoc'
'report 29.08.2016-29.09.2016.html.gdoc'
'Status Report HW Cloud 2020.02.06.gslides'
'Status Report HW Cloud 2020.03.18.gslides'
'TEST_OUTPUT (1).gdoc'
 TEST_OUTPUT.gdoc
 TEST_OUTPUT_t4_test_part1.gdoc
 Untitled.wmv
'Website development.gantter'
'Ваши гитхабы (1).gsheet'
'Ваши гитхабы.gsheet'
'Вопросы к экзамену.gdoc'
'Высокопроизводительный программный комплекс для построения аппроксимации областей.gdoc'
 Генетика.gdoc
'Голос 002.m4a'
'Дз за второй модуль.gsheet'
'Домашнее Задание 20-21.gsheet'
 ИНН.pdf
'Компуктерная фил